# Mapeamento de patentes

## Bibliotecas

In [1]:
import xml.etree.ElementTree as ET
from tqdm import tqdm
import pandas as pd
import os
import requests
import zipfile
import time

## Download das revistas em zip (com tratamento de exceção)

Fonte de dados: http://revistas.inpi.gov.br/rpi/#

In [2]:
#Funcao para verificar se o arquivo zip ja foi baixado anteriormente.
def download_zip(zip_url, output_folder):
    # Nome do arquivo na pasta
    file_name = os.path.basename(zip_url)

    # Caminho completo para o arquivo ZIP
    zip_path = os.path.join(output_folder, file_name)

    # Verifique se o arquivo já existe na pasta
    if os.path.exists(zip_path):
        print(f"Arquivo ZIP '{file_name}' já existe na pasta, pulando o download.")
        return

    # Download do arquivo ZIP diretamente na pasta de saída + TRATAMENTO DE EXCEÇÃO
    max_retries = 3  # Número máximo de tentativas
    for retry in range(max_retries):
        try:
            response = requests.get(zip_url)
            response.raise_for_status()  # Lançar exceção se a resposta não for bem-sucedida

            with open(zip_path, "wb") as zip_file:
                zip_file.write(response.content)

            print(f"Arquivo ZIP salvo em '{zip_path}'")
            break  # Download bem-sucedido, saia do loop de tentativas
        except requests.exceptions.RequestException as e:
            if retry < max_retries - 1:
                if isinstance(e, requests.exceptions.HTTPError) and e.response.status_code == 404:
                    print("Erro: Arquivo não encontrado.")
                    break  # Não é necessário tentar novamente se o arquivo não existe
                else:
                    print(f"Erro ao baixar o arquivo ZIP '{file_name}': {e}")
                    time.sleep(5)  # Aguarde 5 segundos antes de tentar novamente
            else:
                print(f"Erro ao baixar o arquivo ZIP '{file_name}': {e}")
                break

url2 = 2557
while url2 <= 2742:
    zip_url = 'http://revistas.inpi.gov.br/txt/P' + str(url2) + '.zip'
    # Pasta onde você deseja salvar o arquivo ZIP
    output_folder = 'dados_zip'

    # Verifique se a pasta de saída existe e a crie se não existir
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    download_zip(zip_url, output_folder)
    url2 += 1


Arquivo ZIP 'P2557.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2558.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2559.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2560.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2561.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2562.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2563.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2564.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2565.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2566.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2567.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2568.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2569.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2570.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2571.zip' já existe na pasta, pulando o download.
Arquivo ZIP 'P2572.zip' já existe na pas

## Descompactar e organizar os arquivos

In [3]:
def extract_zip_files(output_folder):
    # Certifique-se de que a pasta de saída existe
    if not os.path.exists(output_folder):
        print(f"A pasta de saída '{output_folder}' não existe.")
        return

    # Pasta onde você deseja salvar os arquivos XML extraídos
    xml_folder = 'dados_xml'

    # Verifique se a pasta de saída para os XML existe e a crie se não existir
    if not os.path.exists(xml_folder):
        os.makedirs(xml_folder)

    # Percorre todos os arquivos na pasta de saída
    for root, dirs, files in os.walk(output_folder):
        for file in files:
            if file.lower().endswith(".zip"):
                zip_path = os.path.join(root, file)

                try:
                    # Abra o arquivo ZIP
                    with zipfile.ZipFile(zip_path, "r") as zipf:
                        for filename in zipf.namelist():
                            if filename.startswith("Patente_"):
                                # Extraia o arquivo XML correspondente
                                with zipf.open(filename) as xml_file:
                                    xml_content = xml_file.read()
                                    xml_filename = os.path.join(xml_folder, filename)
                                    with open(xml_filename, "wb") as output_file:
                                        output_file.write(xml_content)

                                # Move o arquivo XML extraído com sucesso para a pasta 'dados_xml'
                                os.rename(xml_filename, os.path.join(xml_folder, filename))
                                print(f"Arquivo XML {filename} extraído com sucesso em '{xml_filename}' e movido para '{xml_folder}'")
                except zipfile.BadZipFile:
                    print(f"Erro ao extrair o arquivo ZIP '{file}': Arquivo ZIP inválido.")
                except Exception as e:
                    print(f"Erro ao processar o arquivo ZIP '{file}': {e}")

# Exemplo de uso
extract_zip_files('dados_zip')

Arquivo XML Patente_2557_07012020.xml extraído com sucesso em 'dados_xml\Patente_2557_07012020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2558_14012020.xml extraído com sucesso em 'dados_xml\Patente_2558_14012020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2559_21012020.xml extraído com sucesso em 'dados_xml\Patente_2559_21012020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2560_28012020.xml extraído com sucesso em 'dados_xml\Patente_2560_28012020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2561_04022020.xml extraído com sucesso em 'dados_xml\Patente_2561_04022020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2562_11022020.xml extraído com sucesso em 'dados_xml\Patente_2562_11022020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2563_18022020.xml extraído com sucesso em 'dados_xml\Patente_2563_18022020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2564_27022020.xml extraído com sucesso em 'dados_xml\Patente_2564_27022020.xml' e movido para 'dad

## Raspagem dos dados

In [4]:
# Definir função para raspagem de um arquivo xml
def raspagem(root):
    # Inicializar listas para armazenar os dados
    data_publicacao = []
    diretoria = []
    numero_revista = []
    codigo = []
    titulo = []
    numero = []
    data_deposito = []
    data_prioridade = []
    cod_prioridade = []
    classificacoes_internacionais = []
    classificacoes_nacionais = []
    titulo_inid = []
    titular_nome = []
    titular_uf = []
    titular_pais = []
    inventor_nome = []

    # Extrair os dados do XML
    for despacho in root.findall(".//despacho[codigo='3.1']"):
        data_publ = root.get('dataPublicacao')
        diret = root.get('diretoria')
        num_rev = root.get('numero')
        cod = despacho.find('codigo').text if despacho.find('codigo') is not None else None
        if cod != None:
            tit = despacho.find('titulo').text if despacho.find('titulo') is not None else None
            processo_patente = despacho.find('processo-patente')
            num = processo_patente.find('.//numero[@inid="21"]').text if processo_patente.find('.//numero[@inid="21"]') is not None else None
            data = processo_patente.find('.//data-deposito[@inid="22"]').text if processo_patente.find('.//data-deposito[@inid="22"]') is not None else None
            data_prio = despacho.find(".//data-prioridade[@inid='31']").text if despacho.find(".//data-prioridade[@inid='31']") is not None else None
            cod_prio1 = despacho.find(".//sigla-pais[@inid='33']").text if despacho.find(".//sigla-pais[@inid='33']") is not None else None
            cod_prio2 = despacho.find(".//numero-prioridade[@inid='32']").text if despacho.find(".//numero-prioridade[@inid='32']") is not None else None
            cod_prio = cod_prio1 + cod_prio2 if (cod_prio1 is not None and cod_prio2 is not None) else None
            internacionais1 = despacho.findall(".//classificacao-internacional[@inid='51']")
            internacionais = '; '.join(ci.text for ci in internacionais1) if internacionais1 else None
            nacionais1 = despacho.findall(".//classificacao-nacional[@inid='52']")
            nacionais = '; '.join(cn.text for cn in nacionais1) if nacionais1 else None
            tit_inid = despacho.find(".//titulo[@inid='54']").text if despacho.find(".//titulo[@inid='54']") is not None else None
            titulares1 = despacho.findall(".//titular[@inid='71']")
            titu_nome = '; '.join(t.find(".//nome-completo").text for t in titulares1) if titulares1 else None
            titu_uf = '; '.join(t.text for t in despacho.findall(".//uf")) if titulares1 else None
            titu_pais = '; '.join(t.text for t in despacho.findall(".//sigla")) if titulares1 else None
            inventores1 = despacho.findall(".//inventor[@inid='72']")
            invent_nome = '; '.join(t.find(".//nome-completo").text for t in inventores1) if inventores1 else None
            # Incluir os dados nas listas
            data_publicacao.append(data_publ)
            diretoria.append(diret)
            numero_revista.append(num_rev)
            codigo.append(cod)
            titulo.append(tit)
            numero.append(num)
            data_deposito.append(data)
            data_prioridade.append(data_prio)
            cod_prioridade.append(cod_prio)
            classificacoes_internacionais.append(internacionais)
            classificacoes_nacionais.append(nacionais)
            titulo_inid.append(tit_inid)
            titular_nome.append(titu_nome)
            titular_uf.append(titu_uf)
            titular_pais.append(titu_pais)
            inventor_nome.append(invent_nome)
        else:
            data_publ = root.get('dataPublicacao')
            diret = root.get('diretoria')
            num_rev = root.get('numero')
            # Incluir os dados na lista
            data_publicacao.append(data_publ)
            diretoria.append(diret)
            numero_revista.append(num_rev)
            codigo.append(None)
            titulo.append(None)
            numero.append(None)
            data_deposito.append(None)
            data_prioridade.append(None)
            cod_prioridade.append(None)
            classificacoes_internacionais.append(None)
            classificacoes_nacionais.append(None)
            titulo_inid.append(None)
            titular_nome.append(None)
            titular_uf.append(None)
            titular_pais.append(None)
            inventor_nome.append(None)

    # Transformar listas em um DataFrame
    df = pd.DataFrame({
        'data_publicacao': data_publicacao,
        'diretoria': diretoria,
        'numero_revista': numero_revista,
        'codigo': codigo,
        'titulo': titulo,
        'numero': numero,
        'data_deposito': data_deposito,
        'data_prioridade': data_prioridade,
        'cod_prioridade': cod_prioridade,
        'classif_internacion': classificacoes_internacionais,
        'classif_nacion': classificacoes_nacionais,
        'titulo_inid': titulo_inid,
        'titular_nome': titular_nome,
        'titular_uf': titular_uf,
        'titular_pais': titular_pais,
        'inventor_nome': inventor_nome
    })

    return df


## Salvar dataframe completo

In [5]:
# Inicializar o dataframe
data = pd.DataFrame()

# Inicializar o timer
start_time = time.time()

# Lista de arquivos
arquivos = os.listdir('dados_xml')

# Inicializar a barra de progresso
with tqdm(total=len(arquivos), desc="Processando arquivos") as pbar:
    for file in arquivos:
        filepath = os.path.join('dados_xml', file)
        tree = ET.parse(filepath)
        root = tree.getroot()
        df = raspagem(root)
        data = pd.concat([data, df], axis=0)
        pbar.update(1)

# Finalizar o timer
end_time = time.time()

# Calcular o tempo decorrido
tempo_decorrido = end_time - start_time

print(f"Tempo decorrido: {tempo_decorrido:.2f} segundos")


Processando arquivos: 100%|██████████| 186/186 [01:21<00:00,  2.28it/s]

Tempo decorrido: 81.78 segundos


In [6]:
data.head(5)

,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,classif_internacion,classif_nacion,titulo_inid,titular_nome,titular_uf,titular_pais,inventor_nome
0,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2013 016644-8,27/06/2013,None,None,B29B 17/00; B29C 43/00; B29K 31/00,B29B 17/00; B29C 43/00,APERFEIÇOAMENTO INTRODUZIDO EM PROCESSO DE REC...,ACM EL ORRA COLCHÕES ME,SP,BR,ABDO CARIM MOUMTEZ EL ORRA
1,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2014 001105-6,16/01/2014,18/01/2013,TW102201214,F16C 11/04; E05F 1/12,F16C 11/04; E05F 1/1215,DOBRADIÇA PARA CONJUNTO DE CAPAS PROTETORAS,"KEYARROW (TAIWAN) CO., LTD.",,TW,"FU-YI CHEN; CHI, LI-LING"
2,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 004929-9,13/03/2017,None,None,H01Q 13/10; H01Q 1/24,H01Q 13/106; H01Q 1/241,"UMA NOVA ANTENA DE MICROFITA TRI-BANDA, DE TAM...",UNIVERSIDADE FEDERAL DO CEARÁ,CE,BR,ANTONIO SERGIO BEZERRA SOMBRA; RONALDO GLAUBER...
3,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 024659-0,17/11/2017,None,None,G01F 1/58; H04W 4/12,G01F 1/58; H04W 4/12,DISPOSITIVO DE MÚLTIPLOS MÓDULOS COM OTIMIZAÇÃ...,CID GRANJA DOS SANTOS,,BR,CID GRANJA DOS SANTOS
4,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 026178-6,05/12/2017,None,None,H04L 29/06; H04L 29/12,H04L 67/42; H04L 61/1558; H04L 29/12141,BEHAVIOR DEFINITION MODEL FOR INTERNET OF THIN...,FUNDAÇÃO UNIVERSIDADE DO AMAZONAS,AM,BR,EDUARDO JAMES PEREIRA SOUTO; NAMEDIN PEREIRA T...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24714 entries, 0 to 57
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   data_publicacao      24714 non-null  object
 1   diretoria            24714 non-null  object
 2   numero_revista       24714 non-null  object
 3   codigo               24714 non-null  object
 4   titulo               24714 non-null  object
 5   numero               24714 non-null  object
 6   data_deposito        24714 non-null  object
 7   data_prioridade      5052 non-null   object
 8   cod_prioridade       5052 non-null   object
 9   classif_internacion  24223 non-null  object
 10  classif_nacion       17269 non-null  object
 11  titulo_inid          24714 non-null  object
 12  titular_nome         24714 non-null  object
 13  titular_uf           24714 non-null  object
 14  titular_pais         24714 non-null  object
 15  inventor_nome        24714 non-null  object
dtypes: object(16

## Tratamento dos dados

In [8]:
# Salvar o DataFrame em um arquivo CSV
data.to_csv('data_patentes.csv', index=False)

In [9]:
# Exporte o DataFrame para o arquivo Excel
data.to_excel('data_patentes_excel.xlsx', sheet_name='Planilha Patentes', index=False)